In [1]:
#%matplotlib inline
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/weather_mast/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/MEPS/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/Retrieval_MEPS/')
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math

import createFolder as cF
import calc_date as cd
import plot_sfc_spaghetti_ret as spagh
import save_fig as SF
import get_Haukeli_obs_data as obsDat
import calc_48h_acc as acc
import fill_values as fv
import plot_vertical as pvert

import os

import pandas as pd
import matplotlib as mpl
mpl.style.use('ggplot')

from calc_diff_obs_model import *
from plt_spaghetti_scatter import *
from scipy import stats

In [2]:
year = 2016
month = 12
tid = '00'
#tid = '18'
station = 'Haukeliseter'
savefig = 1 # 1 = yes, 0 = no

figdir_scatter = '../../Figures/MEPS/scatter_MEPS_obs'
form = 'png'
if savefig == 1:
    cF.createFolder('%s' %(figdir_scatter))

In [3]:
#### days 
t = [#'20', 
    '21', '22', '23', '24', '25', '26']#, '27']

In [4]:
def get_all_day_values(var):
    if var == 'PP':
        var_name = 'precipitation_amount_acc'
    elif var == 'SP':
        var_name = 'air_pressure_at_sea_level'
    elif var == 'T2':
        var_name = 'air_temperature_2m'
    elif var == 'WD':
        var_name = 'wind_direction'
    elif var == 'WS':
        var_name = 'wind_speed'

### MEPS
    if var == 'SP' or var == 'PP':
        level = 'sfc'
    elif var == 'T2':
        level = '2m'
    elif var == 'WD' or var == 'WS':
        level = '10m'

    ncdir_var = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)



################################################################
### Haukeli obs.
    txtdir = '../../Data/eklima/'
    txt_filename = 'Haukeli_obs_Dec2016'

# read in the Haukeliseter time file, sorted by daily values
    Haukeli_Date = pd.read_csv('%s/Observations-Date-Hour.txt' %(txtdir),\
                    sep = ';',header=19, skipfooter = 11, engine= 'python')
    DateHour = Haukeli_Date['Date-Hour(UTC)']

# read in the Haukeliseter observation file, sorted by daily values
    Haukeli = pd.read_csv('%s/%s.txt' %(txtdir,txt_filename),\
                    sep = ';',header=18, skipfooter = 2, engine= 'python')

    if var == 'PP':
        obsvar = Haukeli['RR_1'].astype(float)         # Precipitation (1 hour); [mm]
    elif var == 'T2':
        obsvar = Haukeli['TA'].astype(float)           # Air temperature;degC --> Lufttemperatur ved observasjonstiden i 2 m høyde
    elif var == 'SP':
        obsvar = Haukeli['PR'].astype(float)           # PR;Air pressure at sea level;hPa
    elif var == 'WD' or var == 'WS':
        obsvar_DD = Haukeli['DD'].astype(float)             # Wind direction (FF); [degrees]
        obsvar_FF = Haukeli['FF'].astype(float)             # Wind speed (10 meters above ground);m/s

    if var == 'PP' or var == 'T2' or var == 'SP':
        obsvar0, dt_obsvar0, hour_obsvar0, day_obsvar0, month_obsvar0, year_obsvar0 = obsDat.valid_values(obsvar, DateHour)
    elif var == 'WD' or var == 'WS':
        idxFF = np.where(obsvar_FF[:] == -9999)
        idxDD = np.where(obsvar_DD[:] == -9999)

        obsvar_FF[idxFF[0][:]] = np.nan
        obsvar_DD[idxDD[0][:]] = np.nan

        ### calculate the U, V wind component for barb plot
        # http://colaweb.gmu.edu/dev/clim301/lectures/wind/wind-uv.html
        # first calculate the mathematical wind direction in deg
        md_deg = 270 - obsvar_DD
        for k in range(0,md_deg.shape[0]):
            if md_deg[k] <0 :
                md_deg[k] = md_deg[k] +360
        md_rad = math.pi/180. * md_deg
        uwind = obsvar_FF*np.cos(md_rad)
        vwind = obsvar_FF*np.sin(md_rad)

        obsvar_FF_wind, obsvar_DD_wind, \
                obsvar_dt_wind, obsvar_hour_wind, \
                obsvar_day_wind, obsvar_month_wind, \
                obsvar_year_wind = obsDat.valid_values_wind(obsvar_FF, obsvar_DD, DateHour)
        if var == 'WD':
            obsvar0 = obsvar_DD_wind
        elif var == 'WS':
            obsvar0 = obsvar_FF_wind
        
    fn_var = dict()

    if var == 'PP' or var == 'T2' or var == 'SP':
        model_var = dict()
    elif var == 'WD' or var == 'WS':
        model_var_x = dict()
        model_var_y = dict()

        model_var = dict()


    time_var      = dict()
    lead_time_var = dict()
    ini_day       = dict()
    calday        = dict()
    calmon        = dict()

    calendar_day = []
    initialisation_day = []
    calendar_month = []
    calendar_year = []
    initialisation_hh = []

    diff_var = dict()
    Diff_var_0 = []
    Diff_var_1 = []
    Diff_var_2 = []
    Diff_var_3 = []
    Diff_var_4 = []
    Diff_var_5 = []
    Diff_var_6 = []
    Diff_var_7 = []
    Diff_var_8 = []
    Diff_var_9 = []


    all_day_max_var = []
    all_day_min_var = []

    model_var_0 = []
    model_var_1 = []
    model_var_2 = []
    model_var_3 = []
    model_var_4 = []
    model_var_5 = []
    model_var_6 = []
    model_var_7 = []
    model_var_8 = []
    model_var_9 = []

    obs_var = []

    for day in t:

        if var == 'T2' or var == 'SP' or var == 'WD' or var == 'WS':
            ### connect variable data for two days
            df1 = obsvar0[int(day)-1]
            df2 = obsvar0[int(day)]
            Variable_obs = np.concatenate((df1,df2),axis=0)
            df3 = obsvar0[int(day)+1][0]
            Variable_obs = np.append(Variable_obs,df3)






        for ens_memb in range(0,10):
            fn_var[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir_var,year,month,day,tid,ens_memb))

            time_var[ens_memb] = fn_var[ens_memb].variables['time']
            time_var[ens_memb] = fv.fill_nan(time_var[ens_memb][:])
            lead_time_var[ens_memb] = np.arange(0,time_var[ens_memb].shape[0])
    # initialisation time for plotting
            ini_day = (datetime.datetime.utcfromtimestamp(time_var[0][0]).day)   # day of initialisation
            hh = (datetime.datetime.utcfromtimestamp(time_var[0][0]).hour)       # first hour of initialisation
            calday, calmon = cd.get_dayname(year, month, ini_day)        
    # Read in the variable   
            if var == 'PP' or var == 'T2' or var == 'SP':
                model_var[ens_memb] = fn_var[ens_memb].variables[var_name][:]
                if var == 'T2':
                    model_var[ens_memb] = model_var[ens_memb]-273.15
                elif var == 'SP':
                    model_var[ens_memb] = model_var[ens_memb]/100
                #substitute missing values with nan 
                model_var[ens_memb] = fv.fill_nan(model_var[ens_memb][:])



            elif var == 'WD' or var == 'WS':
                model_var_x[ens_memb] = fn_var[ens_memb].variables['x_wind_10m'][:]
                model_var_y[ens_memb] = fn_var[ens_memb].variables['y_wind_10m'][:]
                #substitute missing values with nan 
                model_var_x[ens_memb] = fv.fill_nan(model_var_x[ens_memb][:,:])
                model_var_y[ens_memb] = fv.fill_nan(model_var_y[ens_memb][:,:])


                #### calculate wind direction / speed
                if var == 'WD':
                    model_var[ens_memb] = np.arctan2(model_var_y[ens_memb][:,0], model_var_x[ens_memb][:,0])
                    model_var[ens_memb] = 270-np.rad2deg(model_var[ens_memb])  # convert from math to meteo direction and to degrees
                    larger360 = np.where(model_var[ens_memb] >360.)
                    model_var[ens_memb][larger360] = model_var[ens_memb][larger360]-360
                elif var == 'WS':
                    model_var[ens_memb] = np.sqrt((model_var_x[ens_memb][:,0])**2 + (model_var_y[ens_memb][:,0])**2) 

    #### calculate every 60 min the accumulation over 48 hours
        if var == 'PP':
            Variable_obs = acc.accumulation_dt60_for48h(obsvar0, ini_day)
    ################

    #### calculate the ensemble mean of variable every 3 hours (all 10 ensemble member averaged)
        val = np.zeros(shape=(10,np.asarray(model_var[0])[::3].shape[0]))
        model_var_std = []
        model_var_mean = []
        for ens_memb in range(0,10):
            if var == 'PP' or var == 'T2' or var == 'SP':
                val[ens_memb] = np.asarray(model_var[ens_memb])[::3,0]
            elif var == 'WD' or var == 'WS':
                val[ens_memb] = np.asarray(model_var[ens_memb])[::3]

        ### std, mean
        model_var_std.append(np.nanstd(val,axis=0,ddof=1))
        model_var_mean.append(np.mean(val,axis=0))

        time_EM_mean = lead_time_var[0][0:np.asarray(model_var[0]).shape[0]:3]

    #    time_model_var_mean = lead_time_var[0][~np.isnan(model_var_mean).any(axis=0)]
     #   model_var_mean = np.asarray(model_var_mean)[~np.isnan(model_var_mean)]
    ################    

    ### save calendar day etc. for scatter plot
        calendar_day.append(calday)
        initialisation_day.append(ini_day)
        calendar_month.append(calmon)
        calendar_year.append(year)
        initialisation_hh.append(hh) 
    ########


    #### calculate the difference between obs and model
        diff_var, Diff_var_0, \
            Diff_var_1, Diff_var_2, Diff_var_3, \
            Diff_var_4, Diff_var_5, Diff_var_6, \
            Diff_var_7, Diff_var_8, Diff_var_9  = calc_diff(diff_var, \
                                                           model_var, Variable_obs, Diff_var_0,\
                                                           Diff_var_1, Diff_var_2, Diff_var_3, \
                                                           Diff_var_4, Diff_var_5, Diff_var_6, \
                                                           Diff_var_7, Diff_var_8, Diff_var_9,var=var)

        if var == 'WD':
            for ens_memb in range(0,10):
                if (diff_var[ens_memb] < -180).any() == True:
                    diff_var[ens_memb][np.where(diff_var[ens_memb] < -180)] = \
                    diff_var[ens_memb][np.where(diff_var[ens_memb] < -180)]+360

                elif (diff_var[ens_memb] > 180).any() == True:
                    diff_var[ens_memb][np.where(diff_var[ens_memb] > 180)] = \
                        diff_var[ens_memb][np.where(diff_var[ens_memb] > 180)]-360

    #########################################
    ### save daily variable
        if var == 'PP' or var == 'T2' or var == 'SP':
            model_var_0.append(model_var[0][:,0])
            model_var_1.append(model_var[1][:,0])
            model_var_2.append(model_var[2][:,0])
            model_var_3.append(model_var[3][:,0])
            model_var_4.append(model_var[4][:,0])
            model_var_5.append(model_var[5][:,0])
            model_var_6.append(model_var[6][:,0])
            model_var_7.append(model_var[7][:,0])
            model_var_8.append(model_var[8][:,0])
            model_var_9.append(model_var[9][:,0])
        elif var == 'WD' or var == 'WS':
            model_var_0.append(model_var[0])#[:,0])
            model_var_1.append(model_var[1])#[:,0])
            model_var_2.append(model_var[2])#[:,0])
            model_var_3.append(model_var[3])#[:,0])
            model_var_4.append(model_var[4])#[:,0])
            model_var_5.append(model_var[5])#[:,0])
            model_var_6.append(model_var[6])#[:,0])
            model_var_7.append(model_var[7])#[:,0])
            model_var_8.append(model_var[8])#[:,0])
            model_var_9.append(model_var[9])#[:,0])

        obs_var.append(Variable_obs)

    #########################################    
        if hh < 10:
            hh = '0%s' %hh
        else:
            hh = hh
    #########################################
        for ens_memb in range(0,10):
            if var == 'PP' or var == 'T2' or var == 'SP':
                lead_time_var[ens_memb] = lead_time_var[ens_memb][~np.isnan(model_var[ens_memb][:,0])]
            elif var == 'WD' or var == 'WS':
                lead_time_var[ens_memb] = lead_time_var[ens_memb][~np.isnan(model_var[ens_memb])]
            model_var[ens_memb]     = model_var[ens_memb][~np.isnan(model_var[ens_memb])]



            fn_var[ens_memb].close()
    ###########

    ###################################
    model_var_all_day = dict()

    model_var_all_day[0]=(model_var_0)
    model_var_all_day[1]=(model_var_1)
    model_var_all_day[2]=(model_var_2)
    model_var_all_day[3]=(model_var_3)
    model_var_all_day[4]=(model_var_4)
    model_var_all_day[5]=(model_var_5)
    model_var_all_day[6]=(model_var_6)
    model_var_all_day[7]=(model_var_7)
    model_var_all_day[8]=(model_var_8)
    model_var_all_day[9]=(model_var_9)

    return(model_var_all_day,obs_var);

In [5]:
def plt_precip_vs_wind(precipitation,wind,x,intercept,slope,mask,
                       precipitation_obs,wind_obs,x_obs,intercept_obs,slope_obs,var,r_value,r_obs):
    
    fig = plt.figure(figsize =(10,10))
    ax = fig.add_subplot(111)

    # fine tuning

    # xaxis  Precipitation
    ax.set_xlim(-0.5,95)
    ax.set_xticks(np.arange(0,100,5))
    ax.set_xticklabels([0, '',10,'',20,'',30,'',40,'',50,'',60,'',70,'',80,'',90,''],fontsize = spagh.tick_fs)
    ax.set_xlabel('Precipitation amount [mm]', fontsize=spagh.label_fs)

    if var == 'WD':
        # yaxis WD
        # Horizontal line to show Wind direction
        ax.axhline(90,color=spagh.vert_col, linewidth= 3)
        ax.axhline(180,color=spagh.vert_col, linewidth= 3)
        ax.axhline(270,color=spagh.vert_col, linewidth= 3)
        ax.axhline(360,color=spagh.vert_col, linewidth= 3)
        # yaxis
        ax.set_ylim(-0.5,360.5)
        ax.set_yticks(np.arange(0,361,45)) 
        ax.set_yticklabels(['N', '', 'E', '', 'S','','W','','N'],fontsize = spagh.tick_fs)
        ax.set_ylabel('Wind direction', fontsize=spagh.label_fs) 
    elif var == 'WS':
        # yaxis WS
        ax.set_ylim(-0.5,30.5)
        ax.set_yticks(np.arange(0,32.5,2.5))
        ax.set_yticklabels([0, '', 5, '',10,'',15,'',20,'',25,'',30], fontsize=spagh.label_fs)
        ax.set_ylabel('Wind speed [m$\,$s$^{-1}$]', fontsize=spagh.label_fs)

    ### plot vari

    # MEPS
    ens_memb=0
    ax.scatter(precipitation[ens_memb][mask[ens_memb]],wind[ens_memb][mask[ens_memb]],
                   color='k',alpha=0.7, s=150)#, label='deterministic')
    ax.plot(x[ens_memb], intercept[ens_memb] + slope[ens_memb]*x[ens_memb],  
                    color ='k',alpha=0.7,
            label='y$_%s$ = {:.2f} + {:.2f}x, R = {:.2f}'.format(intercept[ens_memb], slope[ens_memb],
                                                                 r_value[ens_memb]) %ens_memb)

    ens_memb=1
    ax.scatter(precipitation[ens_memb][mask[ens_memb]],wind[ens_memb][mask[ens_memb]],
                   color=spagh.memb_col,alpha=0.7, s=150)#, label='ensemble member')
    ax.plot(x[ens_memb], intercept[ens_memb] + slope[ens_memb]*x[ens_memb],  
                    color =spagh.memb_col,alpha=0.7,
            label='y$_%s$ = {:.2f} + {:.2f}x, R = {:.2f}'.format(intercept[ens_memb], slope[ens_memb],r_value[ens_memb]) %ens_memb)



    for ens_memb in range(2,10):
        ax.scatter(precipitation[ens_memb][mask[ens_memb]],wind[ens_memb][mask[ens_memb]],
                   color=spagh.memb_col,alpha=0.7, s=150, label='_nolegend_')
        ax.plot(x[ens_memb], intercept[ens_memb] + slope[ens_memb]*x[ens_memb],
                 color =spagh.memb_col,alpha=0.7,
                 label='y$_%s$ = {:.2f} + {:.2f}x, R = {:.2f}'.format(intercept[ens_memb], slope[ens_memb],
                                                                     r_value[ens_memb]) %ens_memb)
    # Observation
    ax.scatter(precipitation_obs,wind_obs,
                    alpha=0.7, s=150, label='_nolegend_')
    ax.plot(x_obs, intercept_obs + slope_obs*x_obs,
                 alpha=0.7,
                 label='y%s = {:.2f} + {:.2f}x, R = {:.2f}'.format(intercept_obs, slope_obs,r_obs) %('$_{obs}$'))
    
    lgd = ax.legend(loc='center left',bbox_to_anchor=(1, .5),
            # fancybox=True, shadow=True, #ncol=3, 
              fontsize=spagh.label_fs-4)
    lgd.set_title( title= '21. Dec - 26. Dec')#, fontsize=spagh.label)
    lgd.get_title().set_fontsize(spagh.label_fs-4)



    # tight layout
    plt.tight_layout()
    return(lgd);

In [6]:
#var = 'PP'
#var = 'SP'
#var = 'T2'
#var = 'WD'
#var = 'WS'

PP_model, PP_obs = get_all_day_values('PP')
WS_model, WS_obs = get_all_day_values('WS')
WD_model, WD_obs = get_all_day_values('WD')

/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1423: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:174: RuntimeWarning: invalid value encountered in greater
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:218: RuntimeWarning: invalid value encountered in less
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:221: RuntimeWarning: invalid value encountered in greater
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:219: RuntimeWarning: invalid value encountered in less


In [7]:
PP_model_flat = dict()
WD_model_flat = dict()
x_model = dict()
y_model = dict()
mask_model = dict()
gradient_model = dict()
intercept_model = dict()
r_value = dict()

for ens_memb in range(0,10):
    PP_model_flat[ens_memb] = np.ndarray.flatten(np.asarray(PP_model[ens_memb]))
    WD_model_flat[ens_memb] = np.ndarray.flatten(np.asarray(WD_model[ens_memb]))
    x_model[ens_memb] = np.asarray(PP_model_flat[ens_memb])
    y_model[ens_memb] = np.asarray(WD_model_flat[ens_memb])
    mask_model[ens_memb] = ~np.isnan(x_model[ens_memb]) & ~np.isnan(y_model[ens_memb])

    gradient_model[ens_memb], intercept_model[ens_memb], r_value[ens_memb], p_value, std_err = \
                                stats.linregress(x_model[ens_memb][mask_model[ens_memb]],
                                                            y_model[ens_memb][mask_model[ens_memb]])
    
PP_obs_flat = np.ndarray.flatten(np.asarray(PP_obs))
WD_obs_flat = np.ndarray.flatten(np.asarray(WD_obs))

x_obs = np.asarray(PP_obs_flat)
y_obs = np.asarray(WD_obs_flat)
mask_obs = ~np.isnan(x_obs) & ~np.isnan(y_obs)
gradient_obs, intercept_obs, r_obs, p_value, std_err = stats.linregress(x_obs,y_obs)

In [8]:
### PLOT Precip vs WD ####
lgd = plt_precip_vs_wind(PP_model_flat,WD_model_flat,x_model,intercept_model,gradient_model,mask_model,
                   PP_obs_flat, WD_obs_flat, x_obs, intercept_obs, gradient_obs,'WD',r_value,r_obs)

if savefig == 1:
    fig_name = 'precip_vs_WD.png'
    plt.savefig('%s/%s' %(figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir_scatter,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

plot saved: ../../Figures/MEPS/scatter_MEPS_obs/precip_vs_WD.png


In [9]:
PP_model_flat = dict()
WS_model_flat = dict()
x_model = dict()
y_model = dict()
mask_model = dict()
gradient_model = dict()
intercept_model = dict()
r_value = dict()

for ens_memb in range(0,10):
    PP_model_flat[ens_memb] = np.ndarray.flatten(np.asarray(PP_model[ens_memb]))
    WS_model_flat[ens_memb] = np.ndarray.flatten(np.asarray(WS_model[ens_memb]))
    x_model[ens_memb] = np.asarray(PP_model_flat[ens_memb])
    y_model[ens_memb] = np.asarray(WS_model_flat[ens_memb])
    mask_model[ens_memb] = ~np.isnan(x_model[ens_memb]) & ~np.isnan(y_model[ens_memb])

    gradient_model[ens_memb], intercept_model[ens_memb], r_value[ens_memb], p_value, std_err = \
                                stats.linregress(x_model[ens_memb][mask_model[ens_memb]],
                                                            y_model[ens_memb][mask_model[ens_memb]])
    
PP_obs_flat = np.ndarray.flatten(np.asarray(PP_obs))
WS_obs_flat = np.ndarray.flatten(np.asarray(WS_obs))

x_obs = np.asarray(PP_obs_flat)
y_obs = np.asarray(WS_obs_flat)
mask_obs = ~np.isnan(x_obs) & ~np.isnan(y_obs)
gradient_obs, intercept_obs, r_obs, p_value, std_err = stats.linregress(x_obs,y_obs)

### PLOT Precip vs WS ####
lgd = plt_precip_vs_wind(PP_model_flat,WS_model_flat,x_model,intercept_model,gradient_model,mask_model,
                   PP_obs_flat, WS_obs_flat, x_obs, intercept_obs, gradient_obs,'WS',r_value,r_obs)
if savefig == 1:
    fig_name = 'precip_vs_WS.png'
    plt.savefig('%s/%s' %(figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir_scatter,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

plot saved: ../../Figures/MEPS/scatter_MEPS_obs/precip_vs_WS.png
